In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt

directory_train = r"PATH train" #REPLACE WITH YOUR PATHS
directory_val = r"PATH VAL"
directory_test = r"PATH TEST"


In [ ]:

transform = transforms.Compose([transforms.Resize((128, 128)),  transforms.ToTensor()])

class MyDataSet(Dataset):
    def __init__(self, directoryOfDataset, transform=None):
        self.data = ImageFolder(directoryOfDataset, transform=transform)

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)

class CNN_FF(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) # 3 * 128 * 128 => conv1 :6 * 124 * 124 => pool: 6 * 62 * 62 => conv2: 16*58*58 => pool2: 16*29*29
        self.pool = nn.MaxPool2d(2, 2) #
        self.conv2 = nn.Conv2d(6, 16, 5) #
        self.fc1 = nn.Linear(16*29*29, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 53)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:

dataset_train = MyDataSet(directory_train, transform=transform)
dataloader_train = DataLoader(dataset_train, batch_size=32, shuffle=True)

dataset_val = MyDataSet(directory_val, transform=transform)
dataloader_val = DataLoader(dataset_val, batch_size=32, shuffle=False)
dataset_test = MyDataSet(directory_test, transform=transform)
dataloader_test = DataLoader(dataset_test, batch_size=32, shuffle=False)


model = CNN_FF()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0

    for images, labels in dataloader_train:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(dataloader_train):.4f}')

def evaluate_model(loader):
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total


val_accuracy = evaluate_model(dataloader_val)
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

test_accuracy = evaluate_model(dataloader_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
